In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

- 데이터 경로 변수

In [2]:
# DATA_PATH = "/content/drive/MyDrive/파이널프로젝트/data/"
# DATA_PATH

In [3]:
DATA_PATH = "data/"
DATA_PATH

'data/'

- 시드값

In [4]:
SEED = 42

- 데이터 불러오기

In [5]:
import pandas as pd
import numpy as np

train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터

train_tr.shape , train_target.shape , test_tr.shape , submit.shape

((523105, 7), (14940, 2), (441196, 7), (12225, 2))

- 공통 피처 파일 불러오기

In [6]:
train_ft = pd.read_csv(f"{DATA_PATH}train_common.csv") # 학습 데이터(피처)
test_ft = pd.read_csv(f"{DATA_PATH}test_common.csv") # 테스트 데이터(피처)

train_ft.shape , test_ft.shape

((14940, 2265), (12225, 2265))

# 결측치 처리

In [7]:
mask = train_ft.isnull().sum() > 0
train_ft.isnull().sum()[mask]

충동구매비율    21
dtype: int64

In [8]:
mask = test_ft.isnull().sum() > 0
test_ft.isnull().sum()[mask]

충동구매비율    24
dtype: int64

In [9]:
train_ft = train_ft.replace([float('inf'), float('-inf')], 0)
test_ft = test_ft.replace([float('inf'), float('-inf')], 0)

In [10]:
train_ft["구매금액_skew"] = train_ft["구매금액_skew"].fillna(0)
train_ft["구매금액_kurtosis"] = train_ft["구매금액_kurtosis"].fillna(0)
test_ft["구매금액_skew"] = test_ft["구매금액_skew"].fillna(0)
test_ft["구매금액_kurtosis"] = test_ft["구매금액_kurtosis"].fillna(0)

In [11]:
train_ft["구매금액표준편차"] = train_ft["구매금액표준편차"].fillna(0)
test_ft["구매금액표준편차"] = test_ft["구매금액표준편차"].fillna(0)

In [12]:
train_ft = train_ft.fillna(0)
test_ft = test_ft.fillna(0)

# 특성 공학(Feature Engineering)

- ID 변수 제외

In [13]:
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 2264), (12225, 2264))

- 추가 피처 만들어 보기

In [14]:
cols = [ col for col in train_ft.columns if col.startswith("pivot_cnt_") ]

In [15]:
train_ft["중분류별_구매횟수_std"] = train_ft[cols].std(axis=1)
train_ft["중분류별_구매횟수_skew"] = train_ft[cols].skew(axis=1)
train_ft["중분류별_구매횟수_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["중분류별_구매횟수_std"] = test_ft[cols].std(axis=1)
test_ft["중분류별_구매횟수_skew"] = test_ft[cols].skew(axis=1)
test_ft["중분류별_구매횟수_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2267), (12225, 2267))

## Feature Encoding

In [16]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

첫구매날짜      337
마지막구매날짜    334
주구매지점        4
주구매_중분류    233
주구매_대분류     28
고객등급         3
dtype: int64

In [17]:
%pip install category_encoders

Note: you may need to restart the kernel to use updated packages.


In [18]:
import category_encoders as ce

In [19]:
enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft[["주구매지점","주구매_대분류"]])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft[["주구매지점","주구매_대분류"]])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 2299), (12225, 2299))

In [20]:
enc = ce.count.CountEncoder()
train_ft["주구매_중분류_cnt"] = enc.fit_transform(train_ft[["주구매_중분류"]])
test_ft["주구매_중분류_cnt"] = enc.transform(test_ft[["주구매_중분류"]])

train_ft.shape, test_ft.shape

((14940, 2300), (12225, 2300))

- 문자열 피처 삭제

In [21]:
cols

['첫구매날짜', '마지막구매날짜', '주구매지점', '주구매_중분류', '주구매_대분류', '고객등급']

In [22]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 2294), (12225, 2294))

In [23]:
train_ft.select_dtypes("object").columns , test_ft.select_dtypes("object").columns

(Index([], dtype='object'), Index([], dtype='object'))

## Feature Scaling

In [24]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [25]:
train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)
train_ft.head()

c:\Users\kwon3\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\kwon3\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\extmath.py:1142: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
c:\Users\kwon3\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\extmath.py:1162: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


,총방문일수,구매주기,주말방문일수,일별평균구매건수,평균구매요일,주구매요일,월요일_구매비율,화요일_구매비율,수요일_구매비율,목요일_구매비율,...,주구매_대분류_20,주구매_대분류_21,주구매_대분류_22,주구매_대분류_23,주구매_대분류_24,주구매_대분류_25,주구매_대분류_26,주구매_대분류_27,주구매_대분류_28,주구매_중분류_cnt
0,-0.442360,0.200876,-0.363995,-0.285092,0.041647,-0.468522,-0.341447,-0.866959,1.298616,0.455780,...,-0.173175,-0.163072,-0.133603,-0.064554,-0.074744,-0.090737,-0.127775,-0.084533,-0.038402,-0.806829
1,0.202134,-0.397875,-0.569485,0.481886,-0.798213,0.118801,-0.503098,0.897414,0.410353,0.701350,...,-0.173175,-0.163072,-0.133603,-0.064554,-0.074744,-0.090737,-0.127775,-0.084533,-0.038402,-0.662367
2,2.028201,-0.896835,1.485416,0.168122,-0.785470,-1.643170,0.938828,0.345647,-0.113721,-0.244096,...,-0.173175,-0.163072,-0.133603,-0.064554,-0.074744,-0.090737,-0.127775,-0.084533,-0.038402,-0.917799
3,3.907976,-1.046522,3.129336,0.577758,0.100820,0.706125,-0.275913,-0.102253,0.153478,0.125255,...,-0.173175,-0.163072,-0.133603,-0.064554,-0.074744,-0.090737,-0.127775,-0.084533,-0.038402,1.409084
4,0.416966,-0.597459,0.457965,-0.484507,1.021483,0.706125,-0.260622,-0.492698,-0.403887,-0.587894,...,-0.173175,-0.163072,-0.133603,-0.064554,-0.074744,-0.090737,-0.127775,-0.084533,-0.038402,0.098431


# 정답 데이터

In [26]:
target = train_target["target"]
target

0        1.0
1        1.0
2        0.0
3        0.0
4        0.0
        ... 
14935    0.0
14936    0.0
14937    0.0
14938    1.0
14939    1.0
Name: target, Length: 14940, dtype: float64

# cv 점수 확인해보기

In [27]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

In [28]:
from lightgbm import LGBMClassifier

model = LGBMClassifier(random_state=SEED)
scores = cross_val_score(model, train_ft, target, cv=cv, scoring="f1_macro", n_jobs=-1)
np.mean(scores)

0.7217821100048953

# 모델 학습

In [29]:
model = LGBMClassifier(random_state=SEED)
model.fit(train_ft,target)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 5874, number of negative: 9066
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.031912 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 32520
[LightGBM] [Info] Number of data points in the train set: 14940, number of used features: 1604
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.393173 -> initscore=-0.433995
[LightGBM] [Info] Start training from score -0.433995


LGBMClassifier(random_state=42)

# 테스트 데이터 예측

In [30]:
pred = model.predict(test_ft)
pred

array([0., 0., 0., ..., 0., 0., 0.])

In [31]:
# pred = model.predict_proba(test_ft)[:,1]
# pred

# 평가를 위한 제출 파일 생성
- 예측 결과를 target 컬럼에 넣어 csv 파일로 저장후에 제출한다.

In [32]:
submit

,ID,target
0,test_0,0.5
1,test_1,0.5
2,test_2,0.5
3,test_3,0.5
4,test_4,0.5
...,...,...
12220,test_12220,0.5
12221,test_12221,0.5
12222,test_12222,0.5
12223,test_12223,0.5


In [33]:
submit["target"] = pred
submit

,ID,target
0,test_0,0.0
1,test_1,0.0
2,test_2,0.0
3,test_3,1.0
4,test_4,0.0
...,...,...
12220,test_12220,1.0
12221,test_12221,0.0
12222,test_12222,0.0
12223,test_12223,0.0


- 예측 결과를 csv 파일로 저장하여 제출

In [34]:
submit.to_csv(f"{DATA_PATH}submit.csv",index=False)